# SERPENTINE - DGEA & GSEA

## Differential Gene Expression Analysis (DGEA)

replicate = patient
donor = sample
label = timepoint

### Set Up Environment

In [ ]:
from __future__ import annotations

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import anndata as ad
import sc_toolbox
#import pertpy 
import decoupler
import seaborn.objects as so

import session_info
import os

In [ ]:
sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=200)
sc.set_figure_params(figsize=(4, 4))

In [ ]:
# Filtering warnings from current version of matplotlib
import warnings

warnings.filterwarnings(
    "ignore", message=".*Parameters 'cmap' will be ignored.*", category=UserWarning
)
warnings.filterwarnings(
    "ignore", message="Tight layout not applied.*", category=UserWarning
)

In [ ]:
# Setting up R dependencies
import anndata2ri
import rpy2
import logging
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro
from rpy2.robjects import r
import random

In [ ]:
rpy2.__version__

In [ ]:
# set up dirs
work_dir = "/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/"
fig_dir = os.path.join(work_dir, "figures", "combined", "main_analysis", "final_figures/")

In [ ]:
# read anndata object
adata = sc.read(os.path.join(work_dir, "data", "outputdata", "combined", "Combined_SCR_CO2_TCR_full-integrated_annot_22-03-24.h5ad"))

In [ ]:
anndata2ri.activate()
%reload_ext rpy2.ipython

In [ ]:
%%R
suppressPackageStartupMessages({
    library(SingleCellExperiment)
})
library(edgeR)
#library(MAST)

### Pseudobulk

Since edgeR was introduced as a method for DE analysis for bulk data, we first need to create pseudobulk samples from our single-cell dataset. For each patient we create 1 pseudobulk sample per cell type by aggregating the cell from each subpopulation and taking the mean gene expression within that subpopulation.

In [ ]:
# Since we need to create pseudobulks for each patient-condition combination, we first need to create such a column by concatenating replicate and label.
adata.obs["DGEA_sample"] = [
    f"P{rep}_{l}" for rep, l in zip(adata.obs["sample"], adata.obs["timepoint"])
]
adata.obs["DGEA_sample"]

In [ ]:
# We need to clean up the cell type names, i.e. replace spaces with underscores and remove + symbols, to avoid Python to R conversion issues.
adata.obs["cell_type"] = adata.obs["Annotation_1.0"]
adata.obs["cell_type"] = [ct.replace(" ", "_") for ct in adata.obs["cell_type"]]
adata.obs["cell_type"] = [ct.replace("+", "") for ct in adata.obs["cell_type"]]
adata.obs["cell_type"]

In [ ]:
# We need to set categorical metadata to be indeed categorical to create pseudobulks.
adata.obs["patient"] = adata.obs["patient"].astype("category")
adata.obs["timepoint"] = adata.obs["timepoint"].astype("category")
adata.obs["DGEA_sample"] = adata.obs["DGEA_sample"].astype("category")
adata.obs["cell_type"] = adata.obs["cell_type"].astype("category")

Now, let’s define the function we need to aggregate single cells into pseudo-replicates:

*aggregate_and_filter* is a function that creates an AnnData object with one pseudo-replicate for each donor for a specified subpopulation from the original single-cell AnnData object. Here, we also filter out donors that have fewer than 30 cells for the specified population.

by changing the *replicates_per_patient* parameter, several (n) pseudo-replicates can be created for each sample; cells are then split into n subsets of roughly equal sizes.

In [ ]:
NUM_OF_CELL_PER_DONOR = 30


def aggregate_and_filter(
    adata,
    cell_identity,
    donor_key="DGEA_sample", #DGEA_sample
    condition_key="timepoint",
    cell_identity_key="cell_type",
    obs_to_keep=[],  # which additional metadata to keep, e.g. gender, age, etc.
    replicates_per_patient=1,
):
    # subset adata to the given cell identity
    adata_cell_pop = adata[adata.obs[cell_identity_key] == cell_identity].copy()
    # check which donors to keep according to the number of cells specified with NUM_OF_CELL_PER_DONOR
    size_by_donor = adata_cell_pop.obs.groupby([donor_key]).size()
    donors_to_drop = [
        donor
        for donor in size_by_donor.index
        if size_by_donor[donor] <= NUM_OF_CELL_PER_DONOR
    ]
    if len(donors_to_drop) > 0:
        print("Dropping the following samples:")
        print(donors_to_drop)
    df = pd.DataFrame(columns=[*adata_cell_pop.var_names, *obs_to_keep])

    adata_cell_pop.obs[donor_key] = adata_cell_pop.obs[donor_key].astype("category")
    for i, donor in enumerate(donors := adata_cell_pop.obs[donor_key].cat.categories):
        print(f"\tProcessing donor {i+1} out of {len(donors)}...", end="\r")
        if donor not in donors_to_drop:
            adata_donor = adata_cell_pop[adata_cell_pop.obs[donor_key] == donor]
            # create replicates for each donor
            indices = list(adata_donor.obs_names)
            random.shuffle(indices)
            indices = np.array_split(np.array(indices), replicates_per_patient)
            for i, rep_idx in enumerate(indices):
                adata_replicate = adata_donor[rep_idx]
                # specify how to aggregate: sum gene expression for each gene for each donor and also keep the condition information
                agg_dict = {gene: "sum" for gene in adata_replicate.var_names}
                for obs in obs_to_keep:
                    agg_dict[obs] = "first"
                # create a df with all genes, donor and condition info
                df_donor = pd.DataFrame(adata_replicate.X.A)
                df_donor.index = adata_replicate.obs_names
                df_donor.columns = adata_replicate.var_names
                df_donor = df_donor.join(adata_replicate.obs[obs_to_keep])
                # aggregate
                df_donor = df_donor.groupby(donor_key).agg(agg_dict)
                df_donor[donor_key] = donor
                df.loc[f"donor_{donor}_{i}"] = df_donor.loc[donor]
    print("\n")
    # create AnnData object from the df
    adata_cell_pop = sc.AnnData(
        df[adata_cell_pop.var_names], obs=df.drop(columns=adata_cell_pop.var_names)
    )
    return adata_cell_pop

We also need to define a separate function to fit an edgeR GLM:

fit_model takes a SingleCellExperiment object as input, creates the design matrix and outputs the fitted GLM. We also output the edgeR object of class DGEList to do some exploratory data analysis (EDA).

In [ ]:
%%R
fit_model <- function(adata_){
    # create an edgeR object with counts and grouping factor
    y <- DGEList(assay(adata_, "X"), group = colData(adata_)$timepoint)
    # filter out genes with low counts
    print("Dimensions before subsetting:")
    print(dim(y))
    print("")
    keep <- filterByExpr(y)
    y <- y[keep, , keep.lib.sizes=FALSE]
    print("Dimensions after subsetting:")
    print(dim(y))
    print("")
    # normalize
    y <- calcNormFactors(y)
    # create a vector that is a concatenation of condition and cell type that we will later use with contrasts
    group <- paste0(colData(adata_)$timepoint, ".", colData(adata_)$cell_type)
    replicate <- colData(adata_)$patient# replicate
    #patient <- colData(adata_)$patient
    # create a design matrix: here we have multiple donors so also consider that in the design matrix
    design <- model.matrix(~ 0 + group + replicate)
    print(design)
    # estimate dispersion
    y <- estimateDisp(y, design = design)
    # fit the model
    fit <- glmQLFit(y, design)
    return(list("fit"=fit, "design"=design, "y"=y))
}

In [ ]:
# vars of metadata to keep
obs_to_keep = ["patient", "cell_type", "timepoint", "DGEA_sample", "sample", "project", "subproject", "ICI_status", "response"]

In [ ]:
# pass rawcounts to edgeR
adata.X = adata.layers["rawcounts"].copy()

In [ ]:
# create anndata object with pseudobulks
# process first cell type separately...
cell_type = adata.obs["cell_type"].cat.categories[0]
print(
    f'Processing {cell_type} (1 out of {len(adata.obs["cell_type"].cat.categories)})...'
)
adata_pb = aggregate_and_filter(adata, cell_type, obs_to_keep=obs_to_keep)
for i, cell_type in enumerate(adata.obs["cell_type"].cat.categories[1:]):
    print(
        f'Processing {cell_type} ({i+2} out of {len(adata.obs["cell_type"].cat.categories)})...'
    )
    adata_cell_type = aggregate_and_filter(adata, cell_type, obs_to_keep=obs_to_keep)
    adata_pb = adata_pb.concatenate(adata_cell_type)

In [ ]:
# We perform very basic EDA on the created pseudo-replicates to check if some patients/pseudobulks are outliers that we need to exclude so as not to bias the DE results. We save the raw counts in the 'counts' layer, then normalize the counts and calculate the PCA coordinates for the normalized pseudobulk counts.
adata_pb.layers['counts'] = adata_pb.X.copy()
sc.pp.normalize_total(adata_pb, target_sum=1e6)
sc.pp.log1p(adata_pb)
sc.pp.pca(adata_pb)

In [ ]:
adata_pb.obs["lib_size"] = np.sum(adata_pb.layers["counts"], axis=1)
adata_pb.obs["log_lib_size"] = np.log(adata_pb.obs["lib_size"].astype(float))

In [ ]:
sc.pl.pca(adata_pb, color=adata_pb.obs, ncols=1, size=300)

In [ ]:
adata_pb.X = adata_pb.layers['counts'].copy()

In [ ]:
# format anndata object to avoid errors
adata_pb.obs = adata_pb.obs.astype(str)
object_columns = adata.obs.select_dtypes(include=['object']).columns
adata.obs[object_columns] = adata.obs[object_columns].astype('category')
from scipy import sparse
adata_pb.X = sparse.csr_matrix(np.array(adata_pb.X, dtype=float))
adata_pb.layers['counts'] = sparse.csr_matrix(np.array(adata_pb.layers['counts'], dtype=float))

In [ ]:
# save anndata object with pseudobulks 
adata_pb.write(os.path.join(work_dir, "data", "outputdata", "combined", "Combined_SCR_CO2_pseudobulks_08-04-24.h5ad"))

In [ ]:
# read anndata object with pseudobulks 
adata_pb = sc.read_h5ad(os.path.join(work_dir, "data", "outputdata", "combined", "Combined_SCR_CO2_pseudobulks_08-04-24.h5ad"))

### Perform DE testing

In [ ]:
adata_pb.obs

In [ ]:
#adata_pb.obs_names = [
#    name.split("_")[1] + "_" + name.split("_")[2] for name in adata_pb.obs_names
#]

In [ ]:
#adata_pb.obs_names

In [ ]:
%%time
%%R -i adata_pb
outs <- fit_model(adata_pb)

In [ ]:
%%R
fit <- outs$fit
y <- outs$y

In [ ]:
%%R
plotMDS(y, col=ifelse(y$samples$group == "stim", "red", "blue"))

In [ ]:
%%R 
colnames(y$design)

In [ ]:
adata_pb

In [ ]:
# Now we use contrasts to perform a quasi-likelihood test for each of our cell types. Because there is no straightforward way to move tables from R to Python from within an R loop we get the results manually for each cell type.

In [ ]:
%%R -i adata_pb -o de_per_cell_type
de_per_cell_type <- list()
cell_types_to_exclude <- c("Liver_Epithelial")#, "replicate03", "replicate10", "replicate02", "replicate08")
for (cell_type in unique(colData(adata_pb)$cell_type)) {
    if (!(cell_type %in% cell_types_to_exclude)) {
        print(cell_type)
        # create contrast for this cell type
        myContrast <- makeContrasts(paste0("groupC02.", cell_type, "-groupSCR.", cell_type), levels = y$design)
        # perform QLF test
        qlf <- glmQLFTest(fit, contrast=myContrast)
        # get all of the DE genes and calculate Benjamini-Hochberg adjusted FDR
        tt <- topTags(qlf, n = Inf)
        # save in the list with the results for all the cell types
        de_per_cell_type[[cell_type]] <- tt$table
    }
}

In [ ]:
#de_per_cell_type

In [ ]:
# save it in .uns 
# get cell types that we ran the analysis for
cell_types = de_per_cell_type.keys()
# add the table to .uns for each cell type
for cell_type in cell_types:
    df = de_per_cell_type[cell_type]
    df["gene_symbol"] = df.index
    df["cell_type"] = cell_type
    sc_toolbox.tools.de_res_to_anndata(
        adata_pb,
        df,
        groupby="cell_type",
        score_col="logCPM",
        pval_col="PValue",
        pval_adj_col="FDR",
        lfc_col="logFC",
        key_added="edgeR_" + cell_type,
    )
#    df.to_csv(f"de_edgeR_{cell_type}.csv")

In [ ]:
de_per_cell_type["B_Cell"]["FDR"]

In [ ]:
# get the table as a pandas dataframe (for this use get_rank_genes_groups_df function)
sc.get.rank_genes_groups_df(adata_pb, group="CD4_T", key="edgeR_CD4_T")[
    :5
]

### Visualization

In [ ]:
FDR = 0.5 #0.5
LOG_FOLD_CHANGE = 1.5


def plot_heatmap(adata, group_key, group_name="cell_type", groupby="timepoint"):
    cell_type = "_".join(group_key.split("_")[1:])
    print(cell_type)
    #print("_____")
    res = sc.get.rank_genes_groups_df(adata, group=cell_type, key=group_key)
    #print(res)
    #print("_____")
    res.index = res["names"].values
    #print(res.index)
    #print("_____")
    #print(res["names"].values)
    #print("_____")
    #print(min(res["pvals"])) # problem with pvals_adj (almost 1)
    #print("_____")
    res = res[(res["pvals_adj"] < FDR) & (abs(res["logfoldchanges"]) > LOG_FOLD_CHANGE)].sort_values(by=["logfoldchanges"])
    #print(res)
    #print("_____")
    #print(f"Plotting {len(res)} genes...")
    markers = list(res.index)
    sc.pl.heatmap(
        adata[adata.obs[group_name] == cell_type].copy(),
        markers,
        groupby=groupby,
        swap_axes=True,
        show_gene_labels=True,
        show=True
    )

In [ ]:
sc.get.rank_genes_groups_df(adata_pb, group="CD4_T", key="edgeR_CD4_T")

In [ ]:
## required formatting for next step
#tempAdata = adata_pb.raw.to_adata()
#tempAdata.var_names = adata_pb.var_names
#adata_pb.raw = tempAdata

In [ ]:
group_keys = ['edgeR_B_Cell', 'edgeR_CAF', 'edgeR_CD4_T', 'edgeR_CD8_T', 'edgeR_Endothelial', 'edgeR_Myeloid', 'edgeR_NK', 'edgeR_Plasma', 'edgeR_Tumor', 'edgeR_pDC']
for group_key in group_keys:
    plot_heatmap(adata_pb, group_key)

In [ ]:
FDR = 0.01
LOG_FOLD_CHANGE = 1.5


def volcano_plot(adata, group_key, group_name="cell_type", groupby="timepoint", title=None):
    cell_type = "_".join(group_key.split("_")[1:])
    result = sc.get.rank_genes_groups_df(adata, group=cell_type, key=group_key).copy()
    result["-logQ"] = -np.log(result["pvals"].astype("float"))
    lowqval_de = result.loc[abs(result["logfoldchanges"]) > LOG_FOLD_CHANGE]
    other_de = result.loc[abs(result["logfoldchanges"]) <= LOG_FOLD_CHANGE]

    fig, ax = plt.subplots()
    sns.regplot(
        x=other_de["logfoldchanges"],
        y=other_de["-logQ"],
        fit_reg=False,
        scatter_kws={"s": 6},
    )
    sns.regplot(
        x=lowqval_de["logfoldchanges"],
        y=lowqval_de["-logQ"],
        fit_reg=False,
        scatter_kws={"s": 6},
    )
    ax.set_xlabel("log2 FC")
    ax.set_ylabel("-log Q-value")

    if title is None:
        title = group_key.replace("_", " ")
    plt.title(title)
    plt.show()

In [ ]:
group_keys = ['edgeR_B_Cell', 'edgeR_CAF', 'edgeR_CD4_T', 'edgeR_CD8_T', 'edgeR_Endothelial', 'edgeR_Myeloid', 'edgeR_NK', 'edgeR_Plasma', 'edgeR_Tumor', 'edgeR_pDC']
for group_key in group_keys:
    volcano_plot(adata_pb, group_key)

In [ ]:
# https://nbisweden.github.io/workshop-scRNAseq/labs/scanpy/scanpy_05_dge.html

In [ ]:
# format anndata object to avoid errors
adata_pb.obs = adata_pb.obs.astype(str)
object_columns_obs = adata_pb.obs.select_dtypes(include='object').columns
adata_pb.obs[object_columns_obs] = adata_pb.obs[object_columns_obs].astype('category')
#from scipy import sparse
#adata_pb.X = sparse.csr_matrix(np.array(adata_pb.X, dtype=float))
#adata_pb.layers['counts'] = sparse.csr_matrix(np.array(adata_pb.layers['counts'], dtype=float))

In [ ]:
group_keys = ['edgeR_B_Cell', 'edgeR_CAF', 'edgeR_CD4_T', 'edgeR_CD8_T', 'edgeR_Endothelial', 'edgeR_Myeloid', 'edgeR_NK', 'edgeR_Plasma', 'edgeR_Tumor', 'edgeR_pDC']
var_keys = ['scores', 'pvals', 'pvals_adj', 'logfoldchanges']
for group_key in group_keys:
    for var_key in var_keys:
        adata_pb.uns[group_key][var_key] = adata_pb.uns[group_key][var_key].astype(float)
        #print(type(adata_pb.uns[group_key][var_key]))

In [ ]:
# save anndata object with DEG info
adata_pb.write(os.path.join(work_dir, "data", "outputdata", "combined", "Combined_SCR_CO2_pseudobulks_DGE_09-04-24.h5ad"))

In [ ]:
# read anndata object with DEG info
adata_pb = sc.read_h5ad(os.path.join(work_dir, "data", "outputdata", "combined", "Combined_SCR_CO2_pseudobulks_DGE_09-04-24.h5ad"))